In [83]:
import pandas as pd

In [84]:
class Account:
    month = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
    season = ["ns", "ns", "ns", "s", "s", "s", "s", "s", "s", "ns", "ns", "ns"]
    user_type = {"la" : "la","lv" : "lv","hv" : "hv"}

    def __init__(self, id, name):
        self.id = id
        self.name = name
        self.contract = self.init_values()
        self.unitcprice = {"s" : self.init_values(),"ns" : self.init_values()}
        self.demand = self.init_values()
        self.data = {}
       
       
    def init_values(self):
        return {"r" : 0, "hp" :0, "ns": 0, "shp" : 0,"op" : 0}
    
    def set_values(self, dict, values):
        dict.update({"r" : values[0], "hp" :values[1], "ns": values[2], "shp" : values[3],"op" : values[4]})
    
    def set_contract(self, values = [0,0,0,0,0]): 
        self.set_values(self.contract, values)

    def set_demand(self, month_index, values = [0,0,0,0,0]): 
        self.set_values(self.demand.setdefault(month_index, {}), values)
        
    def set_unitcontractprice(self, season = "s", values = [0,0,0,0,0]): 
        self.set_values(self.unitcprice.setdefault(season, {}), values)

    def get_demand(self, month_index):
        return self.demand.get(month_index, {})

    def get_unitcontractprice(self, month_index):
        s = self.season[month_index]
        return self.unitcprice.get(s, {})
    
    def caculate_exceed(self, contract, demand):
        exceed = {}
        for key in contract:
            exceed[key] = max(demand[key] - contract[key], 0)

        exceed["hp"] = max(exceed["hp"] - contract["r"], 0)
        exceed["shp"] = max(exceed["shp"] - contract["r"] - contract["hp"], 0)
        exceed["op"] = max(exceed["op"] - contract["r"] - contract["hp"] - contract["shp"], 0)

        exceed["hp"] = max(exceed["hp"] - exceed["r"], 0)
        exceed["shp"] = max(exceed["shp"] - max(exceed["r"],exceed["hp"]), 0)
        exceed["op"] = max(exceed["op"] - max(exceed["r"],exceed["hp"],exceed["shp"]), 0)
      

        return exceed
        
    def calculate_exceed_stage(self, contract, exceed):
        caculate_stage_1 = lambda x, y: 0.1*y if x >= 0.1*y else x
        caculate_stage_2 = lambda x, y: max(x - y, 0)

        exceed_stage_1 = {}
        exceed_stage_2 = {}
        for key in exceed:
            exceed_stage_1[key] = caculate_stage_1(exceed[key], contract[key])
            exceed_stage_2[key] = caculate_stage_2(exceed[key], exceed_stage_1[key])

        exceed_stages = {
            "stage_1" : exceed_stage_1,
            "stage_2" : exceed_stage_2
        }

        return exceed_stages
    
    def calculate_exceed_price(self, exceed_stage, unitcprice, rate = 1):
        exceed_price = {}
        for key in exceed_stage:
            exceed_price[key] = exceed_stage[key] * unitcprice[key] * rate
        return exceed_price

    def calculate_baseprice(self, contract, unitcprice):
        common = max(contract["shp"] + contract["op"] - (contract["r"] + contract["ns"]) * 0.5, 0)
        return contract["r"] * unitcprice["r"] + contract["ns"] * unitcprice["ns"] + contract["hp"] * unitcprice["hp"] + contract["shp"] * unitcprice["shp"] + unitcprice["shp"] * common
    
    def calculate_total(self):
        for month_index in range(12):
            month = self.month[month_index]
            contract = self.contract
            demand = self.get_demand(month_index)
            unitcprice = self.get_unitcontractprice(month_index)
            baseprice = self.calculate_baseprice(contract, unitcprice)

            exceed = self.caculate_exceed(contract, demand)
            exceed_stages = self.calculate_exceed_stage(contract, exceed)

            exceed_price = {
                "stage_1" : self.calculate_exceed_price(exceed_stages["stage_1"], unitcprice, 2),
                "stage_2" : self.calculate_exceed_price(exceed_stages["stage_2"], unitcprice, 3)
            }
            
            exceed_stage_price = {
                "stage_1" : sum(exceed_price["stage_1"].values()),
                "stage_2" : sum(exceed_price["stage_2"].values())
            }

            self.data[month_index] = {
                "month" : month,
                "contract" : contract,
                "demand" : demand,
                "unitcprice" : unitcprice,
                "exceed" : exceed,
                "exceed_stages" : exceed_stages,
                "exceed_price" : exceed_price,
                "exceed_stage_price" : exceed_stage_price,
                "baseprice" : baseprice,
                "totalprice" : baseprice +  exceed_stage_price["stage_1"] +  exceed_stage_price["stage_2"]
            }


In [85]:
user1 = Account(1000,'C1')
user1.set_contract([300,0,0,0,0])
user1.set_unitcontractprice("s", [223.6,166.9,0,44.7,44.7])
user1.set_unitcontractprice("ns", [166.9,166.9,0,33.3,33.3])
demand = [[68,0,0,62,36],
          [66,0,0,42,37],
          [177,0,0,154,36],
          [278,0,0,179,150],
          [276,0,0,164,39],
          [309,0,0,201,40],
          [357,0,0,215,48],
          [329,0,0,205,174],
          [342,0,0,312,54],
          [326,0,0,234,47],
          [240,0,0,173,92],
          [157,0,0,153,36]]

for month_index in range(12):
   s = user1.season[month_index]
   user1.set_demand(month_index, demand[month_index])

user1.calculate_total()

In [86]:
#return the total price of each month


# create a dataframe to store the contract, demand, unit contract price, base price, exceed stage 1, exceed stage 2, exceed stage 1 price, exceed stage 2 price, exceed price, total price

data = user1.data
# print the data for each month
for month, month_data in data.items():
    print(f"{month}")
    for key, value in month_data.items():
        print(f"{key}: {value}")
    print("")


0
month: Jan
contract: {'r': 300, 'hp': 0, 'ns': 0, 'shp': 0, 'op': 0}
demand: {'r': 68, 'hp': 0, 'ns': 0, 'shp': 62, 'op': 36}
unitcprice: {'r': 166.9, 'hp': 166.9, 'ns': 0, 'shp': 33.3, 'op': 33.3}
exceed: {'r': 0, 'hp': 0, 'ns': 0, 'shp': 0, 'op': 0}
exceed_stages: {'stage_1': {'r': 0, 'hp': 0.0, 'ns': 0.0, 'shp': 0.0, 'op': 0.0}, 'stage_2': {'r': 0, 'hp': 0.0, 'ns': 0.0, 'shp': 0.0, 'op': 0.0}}
exceed_price: {'stage_1': {'r': 0.0, 'hp': 0.0, 'ns': 0.0, 'shp': 0.0, 'op': 0.0}, 'stage_2': {'r': 0.0, 'hp': 0.0, 'ns': 0.0, 'shp': 0.0, 'op': 0.0}}
exceed_stage_price: {'stage_1': 0.0, 'stage_2': 0.0}
baseprice: 50070.0
totalprice: 50070.0

1
month: Feb
contract: {'r': 300, 'hp': 0, 'ns': 0, 'shp': 0, 'op': 0}
demand: {'r': 66, 'hp': 0, 'ns': 0, 'shp': 42, 'op': 37}
unitcprice: {'r': 166.9, 'hp': 166.9, 'ns': 0, 'shp': 33.3, 'op': 33.3}
exceed: {'r': 0, 'hp': 0, 'ns': 0, 'shp': 0, 'op': 0}
exceed_stages: {'stage_1': {'r': 0, 'hp': 0.0, 'ns': 0.0, 'shp': 0.0, 'op': 0.0}, 'stage_2': {'r': 0